In [1]:
import requests
from bs4 import BeautifulSoup as bs
import random
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc
import time
import math
import os
import glob, lxml


In [2]:
# Iniciar navegador con undetected_chromedriver
options = uc.ChromeOptions()
# options.add_argument("--headless")  # Descomenta si no quieres ver el navegador
browser = uc.Chrome(options=options)

# ------------------------------------------
# PARTE 1: EXTRAER TODOS LOS ID DE LOS INMUEBLES
# ------------------------------------------
x = 1
ids = []

while True:
    url = f'https://www.idealista.com/venta-viviendas/madrid/moncloa/pagina-{x}.htm'
    print(f"🕸️ Visitando: {url}")
    browser.get(url)
    time.sleep(5)

    soup = bs(browser.page_source, 'lxml')

    try:
        pagina_actual = int(soup.find('main', {'class': 'listing-items'})
                            .find('div', {'class': 'pagination'})
                            .find('li', {'class': 'selected'}).text)
    except:
        print("🚫 Fin de las páginas o error en la paginación.")
        break

    if x != pagina_actual:
        break

    articles = soup.find('main', {'class': 'listing-items'}).find_all('article')
    for article in articles:
        id_muebles = article.get('data-element-id')
        if id_muebles:
            ids.append(id_muebles)
            print(f"📌 ID encontrado: {id_muebles}")
            time.sleep(random.randint(1, 2))

    x += 1

# Guardar los IDs en un DataFrame
ids_casas_C = pd.DataFrame(ids, columns=['id'])

print(ids_casas_C.head(10))
print("Total IDs encontrados:", ids_casas_C.shape[0])


# ------------------------------------------
# PARTE 3: FUNCIÓN PARA PARSEAR CADA INMUEBLE
# ------------------------------------------
def parsear_inmueble(id_muebles):
    print(f"\n📄 Parseando inmueble: {id_muebles}")
    url = f'https://www.idealista.com/inmueble/{id_muebles}/'
    browser.get(url)
    time.sleep(2)
    soup = bs(browser.page_source, 'lxml')

    try:
        titulo = soup.find('span', {'class': 'main-info__title-main'}).text.strip()
        localizacion = soup.find('span', {'class': 'main-info__title-minor'}).text.strip().split(',')[0]
        precio = int(soup.find('span', {'class': 'txt-bold'}).text.replace('.', '').strip())

        # Características básicas
        c1 = soup.find('div', {'class': 'details-property-feature-one'})
        c1_features = c1.find('div', {'class': 'details-property_features'}) if c1 else None
        caracteristicas_basicas = [li.text.strip() for li in c1_features.find_all('li')] if c1_features else []

        # Características extra
        c2 = soup.find('div', {'class': 'details-property-feature-two'})
        caracteristicas_extra = []
        if c2:
            for block in c2.find_all('div', {'class': 'details-property_features'}):
                caracteristicas_extra.extend([li.text.strip() for li in block.find_all('li')])
                
        c3 = soup.find('div', {'class': 'details-property-feature-two'})
        c3_features = c3.find('div', {'class': 'details-property_features'}) if c3 else None
        edificio = [li.text.strip() for li in c3_features.find_all('li')] if c3_features else []

        ubi = soup.find('div', {'id': 'headerMap'})
        ubicacion_des = [zona.text.strip() for zona in ubi.find_all('li')]

        return {
            'id': id_muebles,
            'titulo': titulo,
            'localización': localizacion,
            'precio': precio,
            'caracteristicas_basicas': caracteristicas_basicas,
            'caracteristicas_extra': caracteristicas_extra,
            'ubicacion': ubicacion_des,
            'edificio': edificio,
        
        }

    except Exception as e:
        print(f"❌ Error al parsear {id_muebles}: {e}")
        return None

# ------------------------------------------
# PARTE 4: PARSEAR TODOS LOS INMUEBLES
# ------------------------------------------
resultados = []

for id_mueble in ids_casas_C['id']:
    resultado = parsear_inmueble(id_mueble)
    if resultado:
        resultados.append(resultado)
    time.sleep(random.randint(1, 2))

# Convertir a DataFrame
df_casas_C = pd.DataFrame(resultados)

# Guardar a CSV final
df_casas_C.to_csv('casa_final_moncloa.csv', index=False, sep=';', encoding='utf-16')

# Cerrar el navegador
browser.quit()

print("✅ Proceso completo. Datos guardados en casa_venta_moncloa.csv")


🕸️ Visitando: https://www.idealista.com/venta-viviendas/madrid/moncloa/pagina-1.htm
📌 ID encontrado: 106702840
📌 ID encontrado: 106751275
📌 ID encontrado: 106611139
📌 ID encontrado: 107346813
📌 ID encontrado: 96292622
📌 ID encontrado: 103342470
📌 ID encontrado: 108856132
📌 ID encontrado: 103478338
📌 ID encontrado: 108890798
📌 ID encontrado: 107514241
📌 ID encontrado: 108565052
📌 ID encontrado: 108614155
📌 ID encontrado: 108546329
📌 ID encontrado: 107688503
📌 ID encontrado: 109180829
📌 ID encontrado: 98954898
📌 ID encontrado: 108853262
📌 ID encontrado: 107975978
📌 ID encontrado: 107351499
📌 ID encontrado: 108219254
📌 ID encontrado: 108494808
📌 ID encontrado: 108784047
📌 ID encontrado: 108668138
📌 ID encontrado: 99290081
📌 ID encontrado: 108320393
📌 ID encontrado: 108666390
📌 ID encontrado: 108510361
📌 ID encontrado: 105965544
📌 ID encontrado: 106969492
📌 ID encontrado: 106713206
📌 ID encontrado: 108100095
📌 ID encontrado: 108760315
🕸️ Visitando: https://www.idealista.com/venta-viviendas

WebDriverException: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=140.0.7339.128)
Stacktrace:
	GetHandleVerifier [0x0xe1d2a3+66419]
	GetHandleVerifier [0x0xe1d2e4+66484]
	(No symbol) [0x0xbf4bd3]
	(No symbol) [0x0xbf2244]
	(No symbol) [0x0xbe5662]
	(No symbol) [0x0xbe70c8]
	(No symbol) [0x0xbe58f8]
	(No symbol) [0x0xbe542c]
	(No symbol) [0x0xbe513a]
	(No symbol) [0x0xbe3040]
	(No symbol) [0x0xbe39eb]
	(No symbol) [0x0xbf874e]
	(No symbol) [0x0xc834c7]
	(No symbol) [0x0xc6101c]
	(No symbol) [0x0xc828a1]
	(No symbol) [0x0xc60e16]
	(No symbol) [0x0xc325ce]
	(No symbol) [0x0xc334a4]
	GetHandleVerifier [0x0x1065ee3+2461619]
	GetHandleVerifier [0x0x1060f66+2441270]
	GetHandleVerifier [0x0xe46242+234258]
	GetHandleVerifier [0x0xe36208+168664]
	GetHandleVerifier [0x0xe3d1ad+197245]
	GetHandleVerifier [0x0xe255f8+100040]
	GetHandleVerifier [0x0xe25792+100450]
	GetHandleVerifier [0x0xe0f74a+10266]
	BaseThreadInitThunk [0x0x765b5d49+25]
	RtlInitializeExceptionChai